In [21]:
"""
En este paso filtramos el dataset de alumnos por el año 2016 y hacemos join con el dataset de escuelas.
Luego eliminamos NULLs y grabamos el file df_final.csv. Pensando en predecir el campo nivel_desemp_matematica, 
lo transofrmamos en dos categorias, SATIFACTORIO con valor 0 (satifactorio y avanzado) y 1 BÁSICO (básico y por debajo de básico).
Para ejecutar este paso deben tener en la carpeta /desafio3/datasets , los datasets app_alumno.csv y app_escuela.csv.
"""

import pandas as pd
df = pd.read_csv('../desafio3/datasets/app_alumno.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_escuelas = pd.read_csv('../desafio3/datasets/app_escuela.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df = df[(df['nivel_desemp_matematica'].notnull()) & (df['year_id'] == 2016)]
df = df.drop(['id','repeticion_secundaria','nivel_desemp_ciencias_sociales','nivel_desemp_ciencias_naturales','year_id'], axis=1)
df = df.dropna(axis=0)
df['nivel_desemp_matematica'] = df['nivel_desemp_matematica'].apply(lambda x: 0 if x in ['satisfactorio','avanzado'] else 1)

#### SE AGREGA JOIN PARA AGREGAR AL DATAFRAME LA CANTIDAD DE ALUMNOS QUE TIENE CADA ESCUELA
df_curso = pd.read_csv('../desafio3/datasets/app_curso.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_curso = df_curso[df_curso['year_id'] == 2016]
df_curso_gr = df_curso.groupby(by='escuela_id')['matricula'].sum()
df_curso_gr = df_curso_gr.reset_index()
df_escuelas = df_escuelas.rename(columns={'id': 'escuela_id'})
df_escuelas = df_escuelas.merge(df_curso_gr, how='inner', on ='escuela_id')
####

df = df.merge(df_escuelas, how='inner', on ='escuela_id')
df = df.drop('icse_cat', axis=1)
df = df.dropna(axis=0)
print('#######SHAPE:','\n',df.shape,'\n','#######','\n')
print('#######VALUE_COUNTS:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
print('#######NULLs:','\n',(df.isnull().sum()/df.shape[0])*100,'\n','#######','\n')
print('#######BALANCE:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
df.to_csv('../desafio3/datasets/df_final.csv')

#######SHAPE: 
 (293509, 32) 
 ####### 

#######VALUE_COUNTS: 
 0    175988
1    117521
Name: nivel_desemp_matematica, dtype: int64 
 ####### 

#######NULLs: 
 sexo                               0.0
indice_socioeconomico              0.0
nivel_desemp_matematica            0.0
nivel_desemp_lengua                0.0
ponderador_lengua                  0.0
ponderador_matematica              0.0
ponderador_ciencias_naturales      0.0
ponderador_ciencias_sociales       0.0
tiene_notebook                     0.0
tiene_pc                           0.0
tiene_tablet                       0.0
tiene_celular                      0.0
tiene_smartphone                   0.0
tiene_consola                      0.0
tiene_smarttv                      0.0
tiene_cable                        0.0
tiene_internet_x                   0.0
repeticion_primaria                0.0
escuela_id                         0.0
nivel_id                           0.0
ambito                             0.0
gestion              

In [7]:
"""
En este paso, levantamos el nuevo dataset y convertimos al tipo de dato category las columnas que consideramos categoricas
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df = pd.read_csv('../desafio3/datasets/df_final.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df = df.drop('Unnamed: 0', axis=1)
for i in df.columns:
    if (df[i].dtype == object) or i in ['nivel_id', 'escuela_id', 'provincia_id', 'departamento_id']:
        df[i] = pd.Categorical(df[i])



In [10]:

"""A continuación ejecutamos modelos de regresión logistica para validar que el dataset final sea adecuado para trabajar .
"""
X = df.drop('nivel_desemp_matematica', axis=1)
y = df['nivel_desemp_matematica']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)


In [11]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns = 'n'):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            #c para tomar columnas categoricas, cualquier otro valor para tomar continuas.
            if self.columns == 'c':
                categorical_columns = [col for col in X.columns if pd.api.types.is_categorical_dtype(df[col])]
                return pd.DataFrame(X[categorical_columns])
            else:
                scal_columns = [col for col in X.columns if (pd.api.types.is_categorical_dtype(df[col]) == False)]
                return pd.DataFrame(X[scal_columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
cat_pipe = make_pipeline(ColumnSelector('c'),OneHotEncoder(handle_unknown='ignore'))
num_pipe = make_pipeline(ColumnSelector('n'),StandardScaler())
union = make_union(num_pipe,
                   cat_pipe)



In [46]:
#Este paso tarda bastante dado que ejecutamos un random grid search.
"""
RESULTADO: 
Best score: 0.739
Best parameters set:
	 log__C: 0.1
	 log__penalty: 'l2'
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     44028
           1       0.69      0.63      0.66     29395

    accuracy                           0.74     73423
   macro avg       0.73      0.72      0.72     73423
weighted avg       0.73      0.74      0.74     73423

0.7373302643585797
"""
union.fit_transform(X)

pipeline = Pipeline([('union', union), 
                     ('log', LogisticRegression(solver='liblinear'))])

parameters = {'log__penalty': ['l1', 'l2'],
              'log__C': [0.01, 0.1, 1, 10, 100]}
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
grid_search = RandomizedSearchCV(pipeline, parameters,cv=folds, n_jobs = 5)
grid_search.fit(pd.DataFrame(X_train, columns=X.columns), y_train)

print("Best score: %0.3f" % grid_search.best_score_) 
print("Best parameters set:" )
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted (parameters . keys()): 
                    print("\t %s: %r" % (param_name, best_parameters[param_name])) 
y_pred = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

Best score: 0.739
Best parameters set:
	 log__C: 0.1
	 log__penalty: 'l2'
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     44028
           1       0.69      0.63      0.66     29395

    accuracy                           0.74     73423
   macro avg       0.73      0.72      0.72     73423
weighted avg       0.73      0.74      0.74     73423

0.7373302643585797
